In [ ]:
import socket
import numpy as np
import cv2 as cv
import threading #to run multiple functions at a time

print('-------------------------------Server-----------------------------')
s = socket.socket()
s.bind(("", 5555))
s.listen()
session, address = s.accept() #accepting request from any server
print(session.recv(2048)) 
ipcamip = 'http://192.168.0.103:8080/video' # using camera from IPCamera App #Launch Server from IPCam
camera = cv.VideoCapture(ipcamip)

def sender():
    while True:
        status, photo = camera.read()
        photo = cv.resize(photo, (640, 480))
        if status:
            session.send(np.ndarray.tobytes(photo)) #converting numpy array to bytes so that we can send it to the receiver
        else: 
            print("Image not found")

def receiver():
    print("Entered Into Live Stream with Client")
    while True:
        data = session.recv(100000000)
        if(data == b'finished'):
            print("Finished")
            camera.release()
            session.close()
        else:
            photo =  np.frombuffer(data, dtype=np.uint8) #receving data in numpy form
            if len(photo) == 640*480*3:
                cv.imshow('From Client', photo.reshape(480, 640, 3))
                if cv.waitKey(1) == 13:
                    session.send(b'finished')
                    camera.release()
                    cv.destroyAllWindows()
                    break
            else:
                print('Frames are getting lost')
                
threading.Thread(target=sender).start()
threading.Thread(target=receiver).start()
